In [1]:
import numpy as np
from matplotlib import pyplot as plt
#%matplotlib inline
import pandas as pd

In [2]:
from tqdm import tqdm_notebook

In [3]:
df = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
df['week_day'] = df.DATE.dt.dayofweek

In [4]:
df.describe(include='all')

,DATE,ATM_ID,CLIENT_OUT,week_day
count,287400,287400.00000,2.874000e+05,287400.000000
unique,958,NaN,NaN,NaN
top,2015-02-25 00:00:00,NaN,NaN,NaN
freq,300,NaN,NaN,NaN
first,2015-01-01 00:00:00,NaN,NaN,NaN
last,2017-08-15 00:00:00,NaN,NaN,NaN
mean,NaN,203.08000,3.681474e+05,3.001044
std,NaN,123.98492,4.288130e+05,2.000786
min,NaN,0.00000,-4.100000e+04,0.000000
25%,NaN,88.75000,3.440000e+04,1.000000


In [5]:
df_pivot = pd.pivot_table(df, values='CLIENT_OUT', index='ATM_ID', columns='DATE', margins=True, margins_name='All')

df.CLIENT_OUT.hist(bins=100)

In [6]:
ids = df.ATM_ID.unique()

In [7]:
df.sort_index()

,DATE,ATM_ID,CLIENT_OUT,week_day
0,2015-01-01,0,91600,3
1,2015-01-02,0,136500,4
2,2015-01-03,0,335400,5
3,2015-01-04,0,379000,6
4,2015-01-05,0,344100,0
5,2015-01-06,0,575300,1
6,2015-01-07,0,158200,2
7,2015-01-08,0,296500,3
8,2015-01-09,0,453400,4
9,2015-01-10,0,415900,5


In [8]:
pd.pivot_table(df, values='CLIENT_OUT', index='DATE', columns='ATM_ID')

ATM_ID,0,2,3,4,5,6,7,8,9,10,...,404,405,406,407,408,413,414,415,418,419
DATE,,,,,,,,,,,,,,,,,,,,,
2015-01-01,91600,0,1200,188400,64900,1700,0,0,3800,46000,...,0,0,0,4600,0,200,0,13500,56100,0
2015-01-02,136500,0,2200,223000,87700,133600,0,0,3200,143600,...,0,300,1300,0,0,104400,100,18200,72600,0
2015-01-03,335400,0,12100,382000,159100,246100,0,0,8900,190300,...,0,4000,16400,17700,0,295900,0,30900,43300,0
2015-01-04,379000,0,18000,459400,155300,244000,0,0,7900,178700,...,0,0,15200,1600,0,267100,0,3300,70200,0
2015-01-05,344100,0,700,306400,175100,208300,17300,356900,5300,155600,...,0,0,8000,8000,0,304500,0,12000,90700,0
2015-01-06,575300,0,3000,199900,119700,274400,3600,0,8500,131600,...,0,1200,0,29200,0,286600,0,6600,107400,0
2015-01-07,158200,3000,18600,259500,137400,254300,100,0,1400,146100,...,0,0,0,0,2700,136700,0,300,14600,0
2015-01-08,296500,14700,19900,190500,115800,442400,9900,313900,42100,159000,...,0,0,10600,5200,27800,272100,0,14900,38000,0
2015-01-09,453400,13300,7400,317500,195800,302600,59000,0,7200,265300,...,0,0,52000,1300,3000,365400,16700,14000,68800,0


for id in ids:
    plt.figure(figsize=(15,10))
    plt.title('ATM_ID {0}'.format(id))
    plt.plot(df[df.ATM_ID == id].DATE, df[df.ATM_ID==id].CLIENT_OUT)
    plt.show()

In [9]:
# The dates to predict
pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']
rows = []
for id in ids:
    mean = df[df.ATM_ID==id].CLIENT_OUT.mean()
    for date in pred_dates:
        rows.append((date, id, mean))
submission = pd.DataFrame(rows, columns = ['DATE', 'ATM_ID', 'CLIENT_OUT'])
submission.to_csv('submission.csv', index=False)

In [18]:
def mean_submission(df):
    res = []
    for id in df.ATM_ID.unique():
        mean = df[df.ATM_ID==id].CLIENT_OUT.mean()
        for date in df.DATE.unique():
            res.append((date, id, mean))
    res = pd.DataFrame(res, columns = ['DATE', 'ATM_ID', 'CLIENT_OUT'])
    return res

## Get last month to test

In [11]:
from datetime import datetime, timedelta
month = 30
d = datetime.today() - timedelta(days=month)

In [12]:
from score_submission import scorer

In [13]:
from functions.date_split import split_month_test
train_df, test_df = split_month_test(df, len(pred_dates), 100)

In [14]:
test_df[test_df.ATM_ID==0]

,DATE,ATM_ID,CLIENT_OUT,week_day
925,2017-07-14,0,593800,4
926,2017-07-15,0,217800,5
927,2017-07-16,0,207500,6
928,2017-07-17,0,472000,0
929,2017-07-18,0,325600,1
930,2017-07-19,0,420300,2
931,2017-07-20,0,368700,3
932,2017-07-21,0,696300,4
933,2017-07-22,0,201900,5
934,2017-07-23,0,157700,6


In [25]:
test_df.dtypes

DATE          datetime64[ns]
ATM_ID                 int64
CLIENT_OUT             int64
week_day               int64
dtype: object

In [24]:
mn = mean_submission(test_df)
mn.dtypes

DATE          datetime64[ns]
ATM_ID                 int64
CLIENT_OUT           float64
dtype: object

In [26]:
scorer(test_df.reset_index(), mean_submission(test_df))

0.5861753307813131

In [27]:
import pickle

In [29]:
model = pickle.load(open("rf_clf.pickle", "rb"))

In [30]:
model.predict(test_df)

TypeError: float() argument must be a string or a number, not 'Timestamp'